In [ ]:
cd ..

In [ ]:
ls

In [1]:
import pandas as pd
import lux

In [2]:
lux.config.default_display="lux"
lux.config.plotting_backend="matplotlib"

- tested short column header "A", "B", working as expected

In [7]:
tmax = pd.read_csv('https://raw.githubusercontent.com/koldunovn/python_for_geosciences/master/DelhiTmax.txt', delimiter=r"\s+", parse_dates=[[0,1,2]], header=None)
tmax.columns = ['Date', 'Temp']
tmax

Button(description='Toggle Pandas/Lux', layout=Layout(top='5px', width='140px'), style=ButtonStyle())

Output()

In [9]:
df = pd.DataFrame({'nPts': {0: 49999, 1: 71174, 2: 101317, 3: 144224, 4: 205303, 5: 292249, 6: 416016, 7: 592198, 8: 842993, 9: 1200000}, 't_heatmap': {0: 0.15121674537658691, 1: 0.1811518669128418, 2: 0.2179429531097412, 3: 0.2787730693817139, 4: 0.3973350524902344, 5: 0.4233138561248779, 6: 0.580251932144165, 7: 0.7928099632263184, 8: 1.0876789093017578, 9: 1.6242818832397459}, 't_color_heatmap': {0: 0.14242982864379886, 1: 0.18866705894470212, 2: 0.1566781997680664, 3: 0.16737699508666992, 4: 0.19900894165039065, 5: 0.2701129913330078, 6: 0.2533812522888184, 7: 0.37183785438537603, 8: 0.3830866813659668, 9: 0.39321017265319824}, 't_bar': {0: 0.01940608024597168, 1: 0.02618718147277832, 2: 0.024693727493286133, 3: 0.029685020446777344, 4: 0.03471803665161133, 5: 0.04173588752746582, 6: 0.04706382751464844, 7: 0.0667569637298584, 8: 0.08260798454284668, 9: 0.1006929874420166}, 't_cbar': {0: 0.04185795783996582, 1: 0.050965070724487305, 2: 0.052091121673583984, 3: 0.0610501766204834, 4: 0.07504606246948242, 5: 0.09924101829528807, 6: 0.10392117500305176, 7: 0.13044309616088867, 8: 0.18137121200561526, 9: 0.2069528102874756}, 't_hist': {0: 0.01846003532409668, 1: 0.018136978149414062, 2: 0.018748998641967773, 3: 0.01876473426818848, 4: 0.02434706687927246, 5: 0.025368213653564453, 6: 0.02530217170715332, 7: 0.02823114395141602, 8: 0.029034852981567383, 9: 0.034781217575073235}, 't_scatter': {0: 0.8179378509521484, 1: 1.0982017517089844, 2: 1.4875690937042236, 3: 2.146117925643921, 4: 3.1123709678649902, 5: 3.92416787147522, 6: 6.097048044204713, 7: 8.803220987319945, 8: 12.054464101791382, 9: 17.740845680236816}, 't_color_scatter': {0: 1.3474130630493164, 1: 1.7953579425811768, 2: 2.6268541812896733, 3: 3.646371126174927, 4: 4.901940107345581, 5: 6.974237203598023, 6: 10.197483777999876, 7: 14.134275913238525, 8: 21.13951110839844, 9: 27.127063989639282}})
pdf = df.melt(id_vars=["nPts"],value_vars=['t_heatmap', 't_color_heatmap', 't_bar', 't_cbar', 't_hist','t_scatter', 't_color_scatter'], var_name="type", value_name='time')
pdf

/Users/caitlynachen/Desktop/frontendlux/tutorial-env/lib/python3.8/site-packages/lux_api-0.2.3-py3.8.egg/lux/executor/PandasExecutor.py:402: UserWarning:
Lux detects that the attribute 'time' may be temporal.
To display visualizations for these attributes accurately, please convert temporal attributes to Pandas Datetime objects using the pd.to_datetime function and provide a 'format' parameter to specify the datetime format of the attribute.
For example, you can convert a year-only attribute (e.g., 1998, 1971, 1982) to Datetime type by specifying the `format` as '%Y'.

Here is a starter template that you can use for converting the temporal fields:
	df['time'] = pd.to_datetime(df['time'], format='<replace-with-datetime-format>')

See more at: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.to_datetime.html
If time is not a temporal attribute, please use override Lux's automatically detected type:
	df.set_data_type({'time':'quantitative'})


Button(description='Toggle Pandas/Lux', layout=Layout(top='5px', width='140px'), style=ButtonStyle())

Output()

In [10]:
df = pd.read_csv("../lux/data/car.csv")
df.set_intent(["Horsepower", "Displacement"])
df

/Users/caitlynachen/Desktop/frontendlux/tutorial-env/lib/python3.8/site-packages/lux_api-0.2.3-py3.8.egg/lux/executor/PandasExecutor.py:402: UserWarning:
Lux detects that the attribute 'Year' may be temporal.
To display visualizations for these attributes accurately, please convert temporal attributes to Pandas Datetime objects using the pd.to_datetime function and provide a 'format' parameter to specify the datetime format of the attribute.
For example, you can convert a year-only attribute (e.g., 1998, 1971, 1982) to Datetime type by specifying the `format` as '%Y'.

Here is a starter template that you can use for converting the temporal fields:
	df['Year'] = pd.to_datetime(df['Year'], format='<replace-with-datetime-format>')

See more at: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.to_datetime.html
If Year is not a temporal attribute, please use override Lux's automatically detected type:
	df.set_data_type({'Year':'quantitative'})


Button(description='Toggle Pandas/Lux', layout=Layout(top='5px', width='140px'), style=ButtonStyle())

Output()

In [5]:
df.recommendation

{'Distribution': [<Vis  (x: BIN( ), y: COUNT(Record)) mark: histogram, score: 2.46 >]}

In [6]:
df

Button(description='Toggle Pandas/Lux', layout=Layout(top='5px', width='140px'), style=ButtonStyle())

Output()

In [4]:
df.exported[0]

LuxWidget(current_vis={'config': 'iVBORw0KGgoAAAANSUhEUgAAAUQAAAEgCAYAAADMjwqKAAAAOXRFWHRTb2Z0d2FyZQBNYXRwbG90…

<Vis  (x: Average Wellbeing
(0-10), y: Inequality-adjusted Wellbeing) mark: scatter, score: 0.9944286229474514 >

In [ ]:

# import altair as alt
# alt.Chart(df).mark_bar().encode(
#     x=alt.X(longtitle, axis=alt.Axis(title=longtitle)),
#     y=alt.Y(df., axis=alt.Axis(title="Record"))
# )
import altair as alt
import pandas as pd
source = pd.DataFrame({
    'a': ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I'],
    'b': [28, 55, 43, 91, 81, 53, 19, 87, 52]
})
chart = alt.Chart(source).mark_bar().encode(
    x=alt.X('a', axis=alt.Axis(labelOverlap=True)),
    y=alt.Y('b')
)
chart.encoding.x.axis.title = 'a...'
chart

In [ ]:
longtitle = 'Students across social identities have meaningful opportunities to demonstrate or access the above learning opportunities.'
longtitle2 = 'The teacher provides supports, encouragement, and opportunities equally well for all students in the class, across all sub-groups.'

In [ ]:
df.columns = df.columns.str.replace(".","")

In [ ]:
df

In [ ]:
df.rename(columns={longtitle:longtitle[:-1],longtitle2:longtitle2[:-1]})

In [ ]:
df.data_model